In [49]:
import requests

def create_hgvs(chrom, pos, ref, alt):
    """HGVS benzeri arama formatı üretir"""
    return f"{chrom}:g.{pos}{ref}>{alt}"

def search_variation_id(hgvs_str):
    """ClinVar'da varyant arayıp sadece Variation ID döndürür"""
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "clinvar",
        "term": hgvs_str,
        "retmode": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    idlist = data.get("esearchresult", {}).get("idlist", [])
    return idlist[0] if idlist else None

# Örnek varyant bilgisi
chrom = "17"
pos = "43124028"
ref = "C"
alt = "T"

# HGVS oluştur ve ID ara
hgvs_str = create_hgvs(chrom, pos, ref, alt)
variation_id = search_variation_id(hgvs_str)

# Sadece Variation ID yazdır
if variation_id:
    print(f"Variation ID: {variation_id}")
else:
    print("❌ ClinVar Variation ID bulunamadı.")


Variation ID: 381467


In [50]:
import requests

variation_id = "381467"
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={variation_id}&retmode=json"

response = requests.get(url)
data = response.json()

print(data)


{'header': {'type': 'esummary', 'version': '0.3'}, 'result': {'uids': ['381467'], '381467': {'uid': '381467', 'obj_type': 'single nucleotide variant', 'accession': 'VCV000381467', 'accession_version': 'VCV000381467.18', 'title': 'NM_007294.4(BRCA1):c.69G>A (p.Glu23=)', 'variation_set': [{'measure_id': '375112', 'variation_xrefs': [{'db_source': 'ClinGen', 'db_id': 'CA059450'}, {'db_source': 'dbSNP', 'db_id': '766004110'}], 'variation_name': 'NM_007294.4(BRCA1):c.69G>A (p.Glu23=)', 'cdna_change': 'c.69G>A', 'aliases': [], 'variation_loc': [{'status': 'current', 'assembly_name': 'GRCh38', 'chr': '17', 'band': '17q21.31', 'start': '43124028', 'stop': '43124028', 'inner_start': '', 'inner_stop': '', 'outer_start': '', 'outer_stop': '', 'display_start': '43124028', 'display_stop': '43124028', 'assembly_acc_ver': 'GCF_000001405.38', 'annotation_release': '', 'alt': '', 'ref': ''}, {'status': 'previous', 'assembly_name': 'GRCh37', 'chr': '17', 'band': '17q21.31', 'start': '41276045', 'stop': 

In [51]:
import requests

# 🔎 Variation ID ile ClinVar'dan veri çek
variation_id = "381467"
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={variation_id}&retmode=json"

response = requests.get(url)
data = response.json()

# 🔍 Veriyi özetle
vid = list(data["result"]["uids"])[0]
info = data["result"][vid]

title = info.get("title", "Yok")
accession = info.get("accession_version", "Yok")
gene = info.get("genes", [{}])[0].get("symbol", "Yok")
variant_type = info.get("obj_type", "Yok")

locations = info.get("variation_set", [])[0].get("variation_loc", [])
grch38 = next((l for l in locations if l["assembly_name"] == "GRCh38"), {})
grch37 = next((l for l in locations if l["assembly_name"] == "GRCh37"), {})
spdi = info["variation_set"][0].get("canonical_spdi", "Yok")

clinical = info.get("germline_classification", {})
clinical_sig = clinical.get("description", "Yok")
review_status = clinical.get("review_status", "Yok")
trait = clinical.get("trait_set", [{}])[0].get("trait_name", "Yok")
last_eval = clinical.get("last_evaluated", "Yok")

allele_freqs = info["variation_set"][0].get("allele_freq_set", [])
freq_str = "\n".join([f"{f['source']}: {f['value']}" for f in allele_freqs]) or "Yok"

molecular = ", ".join(info.get("molecular_consequence_list", [])) or "Yok"

summary = f"""
🔍 Varyant Özeti:
- Title: {title}
- Accession: {accession}
- Gene: {gene}
- Variation Type: {variant_type}

🧬 Genomik Konum:
- GRCh38: Chr {grch38.get('chr')} : {grch38.get('start')}
- GRCh37: Chr {grch37.get('chr')} : {grch37.get('start')}
- SPDI: {spdi}

⚠️ Klinik Bilgi:
- Klinik Sınıflandırma: {clinical_sig}
- İlişkili Hastalık: {trait}
- Değerlendirme: {review_status} ({last_eval})

📊 Allele Frekansları:
{freq_str}

🧬 Moleküler Etki:
{molecular}
"""

print(summary)



🔍 Varyant Özeti:
- Title: NM_007294.4(BRCA1):c.69G>A (p.Glu23=)
- Accession: VCV000381467.18
- Gene: BRCA1
- Variation Type: single nucleotide variant

🧬 Genomik Konum:
- GRCh38: Chr 17 : 43124028
- GRCh37: Chr 17 : 41276045
- SPDI: NC_000017.11:43124027:C:T

⚠️ Klinik Bilgi:
- Klinik Sınıflandırma: Likely benign
- İlişkili Hastalık: Breast-ovarian cancer, familial, susceptibility to, 1
- Değerlendirme: reviewed by expert panel (2017/06/29 00:00)

📊 Allele Frekansları:
The Genome Aggregation Database (gnomAD): 0.00000
The Genome Aggregation Database (gnomAD), exomes: 0.00000
Exome Aggregation Consortium (ExAC): 0.00001
The Genome Aggregation Database (gnomAD): 0.00001
The Genome Aggregation Database (gnomAD), exomes: 0.00001
Trans-Omics for Precision Medicine (TOPMed): 0.00001

🧬 Moleküler Etki:
5 prime UTR variant, synonymous variant, intron variant

